In [3]:
library(ArchR)
library(tidyverse)

In [4]:
getwd()

[1] "/nfs/team205/heart/anndata_objects/Foetal/multiome_ATAC/ArchR"

In [5]:
# Prepare input files
metadata = read.csv('/nfs/team205/heart/anndata_objects/Foetal/metadata/HeartTeamSamples_Mappeddata_20230208.csv')
metadata = metadata %>% filter(Publication=='Foetal')
metadata = metadata %>% filter(status=='Healthy')
metadata = metadata %>% filter(modality=='Multiome-ATAC')
nrow(metadata)

inputFiles = paste0(metadata %>% pull('Mapping_farm'),'/atac_fragments.tsv.gz')
names(inputFiles) = metadata %>% pull('CombinedID_Multiome')
inputFiles

[1] 40

BHF_F_Hea11064670_BHF_F_Hea11031823 
  "/nfs/team205/heart/mapped/cellranger-arc200/BHF_F_Hea11064670_BHF_F_Hea11031823/atac_fragments.tsv.gz" 
                                                                      BHF_F_Hea11064671_BHF_F_Hea11031824 
  "/nfs/team205/heart/mapped/cellranger-arc200/BHF_F_Hea11064671_BHF_F_Hea11031824/atac_fragments.tsv.gz" 
                                                                      BHF_F_Hea11064672_BHF_F_Hea11031825 
  "/nfs/team205/heart/mapped/cellranger-arc200/BHF_F_Hea11064672_BHF_F_Hea11031825/atac_fragments.tsv.gz" 
                                                                      BHF_F_Hea11933664_BHF_F_Hea11596617 
  "/nfs/team205/heart/mapped/cellranger-arc200/BHF_F_Hea11933664_BHF_F_Hea11596617/atac_fragments.tsv.gz" 
                                                                      BHF_F_Hea11933665_BHF_F_Hea11596618 
  "/nfs/team205/heart/mapped/cellranger-arc200/BHF_F_Hea11933665_BHF_F_Hea11596618/atac_fragments.tsv.gz" 
                                                                      BHF_F_Hea11933666_BHF_F_Hea11596619 
  "/nfs/team205/heart/mapped/cellranger-arc200/BHF_F_Hea11933666_BHF_F_Hea11596619/atac_fragments.tsv.gz" 
                                                                      BHF_F_Hea11933667_BHF_F_Hea11596620 
  "/nfs/team205/heart/mapped/cellranger-arc200/BHF_F_Hea11933667_BHF_F_Hea11596620/atac_fragments.tsv.gz" 
                                                                      BHF_F_Hea11933668_BHF_F_Hea11596621 
  "/nfs/team205/heart/mapped/cellranger-arc200/BHF_F_Hea11933668_BHF_F_Hea11596621/atac_fragments.tsv.gz" 
                                                                      BHF_F_Hea11933669_BHF_F_Hea11596622 
  "/nfs/team205/heart/mapped/cellranger-arc200/BHF_F_Hea11933669_BHF_F_Hea11596622/atac_fragments.tsv.gz" 
                                                                      BHF_F_Hea11933670_BHF_F_Hea11596623 
  "/nfs/team205/heart/mapped/cellranger-arc200/BHF_F_Hea11933670_BHF_F_Hea11596623/atac_fragments.tsv.gz" 
                                                                      BHF_F_Hea11933671_BHF_F_Hea11596624 
  "/nfs/team205/heart/mapped/cellranger-arc200/BHF_F_Hea11933671_BHF_F_Hea11596624/atac_fragments.tsv.gz" 
                                                                      BHF_F_Hea11933672_BHF_F_Hea11596625 
  "/nfs/team205/heart/mapped/cellranger-arc200/BHF_F_Hea11933672_BHF_F_Hea11596625/atac_fragments.tsv.gz" 
                                                                      BHF_F_Hea11933673_BHF_F_Hea11596626 
  "/nfs/team205/heart/mapped/cellranger-arc200/BHF_F_Hea11933673_BHF_F_Hea11596626/atac_fragments.tsv.gz" 
                                                                      BHF_F_Hea11933674_BHF_F_Hea11596627 
  "/nfs/team205/heart/mapped/cellranger-arc200/BHF_F_Hea11933674_BHF_F_Hea11596627/atac_fragments.tsv.gz" 
                                                                      BHF_F_Hea11933675_BHF_F_Hea11596628 
  "/nfs/team205/heart/mapped/cellranger-arc200/BHF_F_Hea11933675_BHF_F_Hea11596628/atac_fragments.tsv.gz" 
                                                                      BHF_F_Hea13188316_BHF_F_Hea13187618 
  "/nfs/team205/heart/mapped/cellranger-arc200/BHF_F_Hea13188316_BHF_F_Hea13187618/atac_fragments.tsv.gz" 
                                                                      BHF_F_Hea13188317_BHF_F_Hea13187619 
  "/nfs/team205/heart/mapped/cellranger-arc200/BHF_F_Hea13188317_BHF_F_Hea13187619/atac_fragments.tsv.gz" 
                                                                      BHF_F_Hea13188318_BHF_F_Hea13187620 
  "/nfs/team205/heart/mapped/cellranger-arc200/BHF_F_Hea13188318_BHF_F_Hea13187620/atac_fragments.tsv.gz" 
                                                                      BHF_F_Hea13188324_BHF_F_Hea13187626 
  "/nfs/team205/heart/mapped/cellranger-arc200/BHF_F_Hea13188324_BHF_F_Hea13187626/atac_fragments.tsv.gz" 
    

In [6]:
# before starting a project we must set the ArchRGenome and default threads for parallelization.
# Setting default genome to Hg38.
addArchRGenome("hg38")

Setting default genome to Hg38.



In [7]:
# Setting default number of Parallel threads to 16
addArchRThreads(threads = 5) 

Setting default number of Parallel threads to 5.



## Creat Arrow files

In [6]:
# ArrowFiles <- createArrowFiles(
#   inputFiles = inputFiles,
#   sampleNames = names(inputFiles),
#   minTSS = 4, #Dont set this too high because you can always increase later
#   minFrags = 1000, 
#   addTileMat = TRUE,
#   addGeneScoreMat = TRUE
# )

Using GeneAnnotation set by addArchRGenome(Hg38)!

Using GeneAnnotation set by addArchRGenome(Hg38)!

ArchR logging to : ArchRLogs/ArchR-createArrows-e075303579c-Date-2023-03-08_Time-01-11-16.log
If there is an issue, please report to github with logFile!

Cleaning Temporary Files

2023-03-08 01:11:17 : Batch Execution w/ safelapply!, 0 mins elapsed.



In [8]:
done_files = list.files('/nfs/team205/heart/anndata_objects/Foetal/multiome_ATAC/ArchR',pattern = '.arrow')
inputFiles_forrerun = inputFiles[setdiff(names(inputFiles),unlist(strsplit(done_files,'.arrow')))]
inputFiles_forrerun

BHF_F_Hea11064670_BHF_F_Hea11031823 
"/nfs/team205/heart/mapped/cellranger-arc200/BHF_F_Hea11064670_BHF_F_Hea11031823/atac_fragments.tsv.gz" 
                                                                    BHF_F_Hea11064671_BHF_F_Hea11031824 
"/nfs/team205/heart/mapped/cellranger-arc200/BHF_F_Hea11064671_BHF_F_Hea11031824/atac_fragments.tsv.gz" 
                                                                    BHF_F_Hea11064672_BHF_F_Hea11031825 
"/nfs/team205/heart/mapped/cellranger-arc200/BHF_F_Hea11064672_BHF_F_Hea11031825/atac_fragments.tsv.gz" 
                                                                    BHF_F_Hea11933664_BHF_F_Hea11596617 
"/nfs/team205/heart/mapped/cellranger-arc200/BHF_F_Hea11933664_BHF_F_Hea11596617/atac_fragments.tsv.gz" 
                                                                    BHF_F_Hea11933665_BHF_F_Hea11596618 
"/nfs/team205/heart/mapped/cellranger-arc200/BHF_F_Hea11933665_BHF_F_Hea11596618/atac_fragments.tsv.gz" 
                                                                    BHF_F_Hea11933666_BHF_F_Hea11596619 
"/nfs/team205/heart/mapped/cellranger-arc200/BHF_F_Hea11933666_BHF_F_Hea11596619/atac_fragments.tsv.gz" 
                                                                    BHF_F_Hea11933667_BHF_F_Hea11596620 
"/nfs/team205/heart/mapped/cellranger-arc200/BHF_F_Hea11933667_BHF_F_Hea11596620/atac_fragments.tsv.gz" 
                                                                    BHF_F_Hea11933668_BHF_F_Hea11596621 
"/nfs/team205/heart/mapped/cellranger-arc200/BHF_F_Hea11933668_BHF_F_Hea11596621/atac_fragments.tsv.gz" 
                                                                    BHF_F_Hea11933669_BHF_F_Hea11596622 
"/nfs/team205/heart/mapped/cellranger-arc200/BHF_F_Hea11933669_BHF_F_Hea11596622/atac_fragments.tsv.gz" 
                                                                    BHF_F_Hea11933671_BHF_F_Hea11596624 
"/nfs/team205/heart/mapped/cellranger-arc200/BHF_F_Hea11933671_BHF_F_Hea11596624/atac_fragments.tsv.gz" 
                                                                    BHF_F_Hea11933674_BHF_F_Hea11596627 
"/nfs/team205/heart/mapped/cellranger-arc200/BHF_F_Hea11933674_BHF_F_Hea11596627/atac_fragments.tsv.gz" 
                                                                    BHF_F_Hea11933675_BHF_F_Hea11596628 
"/nfs/team205/heart/mapped/cellranger-arc200/BHF_F_Hea11933675_BHF_F_Hea11596628/atac_fragments.tsv.gz" 
                                                                    BHF_F_Hea13188317_BHF_F_Hea13187619 
"/nfs/team205/heart/mapped/cellranger-arc200/BHF_F_Hea13188317_BHF_F_Hea13187619/atac_fragments.tsv.gz" 
                                                                    BHF_F_Hea13188329_BHF_F_Hea13187631 
"/nfs/team205/heart/mapped/cellranger-arc200/BHF_F_Hea13188329_BHF_F_Hea13187631/atac_fragments.tsv.gz" 
                                                                    BHF_F_Hea13188330_BHF_F_Hea13187632 
"/nfs/team205/heart/mapped/cellranger-arc200/BHF_F_Hea13188330_BHF_F_Hea13187632/atac_fragments.tsv.gz" 
                                                                     7089STDY13216920_BHF_F_Hea13242527 
 "/nfs/team205/heart/mapped/cellranger-arc200/7089STDY13216920_BHF_F_Hea13242527/atac_fragments.tsv.gz" 
                                                                     7089STDY13216922_BHF_F_Hea13242529 
 "/nfs/team205/heart/mapped/cellranger-arc200/7089STDY13216922_BHF_F_Hea13242529/atac_fragments.tsv.gz" 
                                                                     7089STDY13216923_BHF_F_Hea13242530 
 "/nfs/team205/heart/mapped/cellranger-arc200/7089STDY13216923_BHF_F_Hea13242530/atac_fragments.tsv.gz" 
                                                                     7089STDY13216925_BHF_F_Hea13242532 
 "/nfs/team205/heart/mapped/cellranger-arc200/7089STDY13216925_BHF_F_Hea13242532/atac_fragments.tsv.gz" 
                                                                     7089STDY1

In [9]:
ArrowFiles <- createArrowFiles(
  inputFiles = inputFiles_forrerun,
  sampleNames = names(inputFiles_forrerun),
  minTSS = 4, #Dont set this too high because you can always increase later
  minFrags = 1000, 
  addTileMat = TRUE,
  addGeneScoreMat = TRUE
)

Using GeneAnnotation set by addArchRGenome(Hg38)!

Using GeneAnnotation set by addArchRGenome(Hg38)!

ArchR logging to : ArchRLogs/ArchR-createArrows-35a75076e3b-Date-2023-03-08_Time-18-21-49.log
If there is an issue, please report to github with logFile!

Cleaning Temporary Files

2023-03-08 18:21:50 : Batch Execution w/ safelapply!, 0 mins elapsed.

ArchR logging successful to : ArchRLogs/ArchR-createArrows-35a75076e3b-Date-2023-03-08_Time-18-21-49.log



In [ ]:
?createArrowFiles

In [ ]:
getwd()